### To Do / License

- Generate a fake customer profile
- Query the customer past purchases
- Create a marketing profile for the customer
- Run this at scale and fill in the data in the customer_marketing_profile table

```
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################
```

Author: Paul Ramsey, Adam Paternostro


### Initialize

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2

from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
location="us-central1"

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Functions

#### GeminiLLM (Pro 1.0 , Pro 1.5 and Pro Vision 1.0)

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### RunQuery(sql)

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

#### PrettyPrintJson(json_string)

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

### Generate Customer Marketing Profile Data



The code below generates and derives data to populate the `customer_marketing_profile` table. The table is defined as follows:

```sql
CREATE TABLE IF NOT EXISTS `chocolate_ai.customer_marketing_profile`
(
    customer_id                  INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_profile_data        JSON             OPTIONS(description="The raw data we know about a customer."),
    data_beans_profile_data      JSON             OPTIONS(description="Chocolate A.I. data about the customer's purchases and reviews."),
    generated_marketing_insights JSON             OPTIONS(description="The generated marketing insights in JSON format"),
    customer_text_summary        STRING           OPTIONS(description="Text summary of the generated marketing profile"),
)
CLUSTER BY customer_id;
```

The data generator and derivation functions produce output like the following samples:
- customer_profile_data
  ```json
  {"children":"Yes","coffee_preferences":["Sumatra Mandheling","French Press","Late Afternoon"],"content_interaction":["High engagement with coffee brewing tutorials","Clicks on articles about sustainable coffee farming","Regularly watches videos about coffee origins"],"customer_age":36,"customer_id":205,"customer_service_interactions":[{"contact_reason":"Inquiry about a specific coffee bean","resolution_time":"12 hours","sentiment_analysis":"Positive"}],"dietary_preferences":[],"education":"High School Diploma","facebook_bio":"","facebook_engagement":"","facebook_handle":"","instagram_bio":"Exploring the backstreets of Tokyo, one ramen shop at a time.  #TokyoFoodie #TravelBug #CityLife","instagram_engagement":"Active","instagram_handle":"tokyo_wanderer205","interests":["Traveling","Foodie Adventures","Urban Exploration","Photography"],"lifestyle":["Urban","Foodie","Weekender"],"linkedin_bio":"","linkedin_engagement":"","linkedin_handle":"","martial_status":"Single","occupation":"Delivery Driver","solicated_buying_habits":["Weekends","Special Occasions"],"sports":["Baseball","Sumo"],"tiktok_bio":"","tiktok_engagement":"","tiktok_handle":"","twitter_bio":"","twitter_engagement":"","twitter_handle":"","youtube_bio":"","youtube_engagement":"","youtube_handle":""}
  ```
- data_beans_profile_data
  ```json
  {"average_amount_spent_per_order":6.231212415130943,"last_order_date":"2024-09-13T16:26:18.628451Z","latest_review_sentiment":"Positive","most_frequent_purchase_location":7,"negative_review_percentage":22.22222222222222,"neutral_review_percentage":11.11111111111111,"positive_review_percentage":66.66666666666667,"purchase_locations":[17,19,23,44,16,22,1,36,31,35,37,27,46,3,49,48,10,14,20,28,45,33,8,13,24,25,43,50,26,2,5,6,29,30,34,40,41,47,15,12,7,4,9,11,18,32,39,38,21,42],"top_3_favorite_menu_items":[64,155,115],"total_amount_spent":3854.6279999999997,"total_orders":1369,"total_reviews":9}
  ```
- generated_marketing_insights
  ```json
  {"customer_segments":{"Demographic Segmentation":{"Age":"35-44","Education":"High School Graduate","Ethnicity":"Unknown","Family Size":"Unknown","Gender":"Unknown","Generation":"Millennial","Income":"Unknown","Language":"Unknown","Marital Status":"Single","Occupation":"Delivery Driver"},"Geographic Segmentation":{"City":"Unknown","Climate":"Unknown","Country":"Unknown","Population Density":"Unknown","Region":"Unknown","Time Zone":"Unknown","Urban/Rural":"Urban"},"Psychographic Segmentation":{"Attitudes":"Positive","Hobbies":"Photography","Interests":"Travel","Lifestyle":"Social","Motivations":"Unknown","Personality":"Unknown","Social Class":"Unknown","Values":"Unknown"},"Technographic Segmentation":{"Adoption Rate":"Unknown","Browsers":"Unknown","Devices":"Smartphone","Internet Connectivity":"Unknown","Operating Systems":"Unknown","Social Media Platforms":"Instagram","Software":"Unknown","Tech Savviness":"Unknown"},"Value-Based Segmentation":{"Cost-Benefit Analysis":"Unknown","Perceived Value":"Unknown","Price Sensitivity":"Unknown","Willingness to Pay":"Unknown"}}}
  ```
- customer_text_summary
  ```json
  This 36-year-old single male customer is an active Instagram user (@tokyo_wanderer205) who enjoys exploring urban areas, photography, and foodie adventures, particularly in Tokyo. His online behavior shows a keen interest in coffee, evident from his engagement with coffee-related content. He is a regular customer with 1369 orders, totaling $3854.63, and predominantly provides positive reviews.  His most recent purchase was on September 13th, 2024. To effectively target him, leverage his Instagram presence and interest in travel and food, potentially pairing it with urban coffee experiences or promotions related to Japanese coffee culture.
  ```

### Step 1: Create customer_marketing_profile table

In [ ]:
%%bigquery

--DROP TABLE IF EXISTS `chocolate_ai.customer_profile`;

CREATE TABLE IF NOT EXISTS `chocolate_ai.customer_marketing_profile`
(
    customer_id                  INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_profile_data        JSON             OPTIONS(description="The raw data we know about a customer."),
    data_beans_profile_data      JSON             OPTIONS(description="Chocolate A.I. data about the customer's purchases and reviews."),
    generated_marketing_insights JSON             OPTIONS(description="The generated marketing insights in JSON format"),
    customer_text_summary        STRING           OPTIONS(description="Text summary of the generated marketing profile"),
)
CLUSTER BY customer_id;

### Step 2: Generate customer_profile_data JSON

#### Define Generator Function

In [ ]:
def GenerateCustomerMarketingProfile(customer_id, customer_name, customer_yob, customer_inception_date, country_code, city_name):

  import random
  import datetime

  # In the United States, estimates suggest that around 6% of adults and nearly 8% of children (or 1 in 13 children) have food allergies
  dietary_preferences_random_number = random.randint(1, 100)
  customer_age = datetime.datetime.now().year - customer_yob

  employeement_status_array = ["Full-time", "Part-time", "Contract Worker", "Freelancer", "Gig Worker"]
  job_skill_level_array = ["Professional", "Paraprofessional", "Skilled Trades", "Unskilled Labor", "Blue Collar", "White Collar"]
  other_categories_array = ["Management", "Sales", "Executive", "Entrepreneur", "Volunteer", "Professor", "Doctor"]

  # Generate random job profile
  job_profile = random.choice(employeement_status_array) + ", " + random.choice(job_skill_level_array)

  if random.randint(1, 100) < 10:
    job_profile = job_profile + ", " + random.choice(other_categories_array)

  print (f"customer_id: {customer_id}")
  print (f"customer_name: {customer_name}")
  print (f"customer_yob: {customer_yob}")
  print (f"customer_inception_date: {customer_inception_date}")
  print (f"country_code: {country_code}")
  print (f"city_name: {city_name}")
  print (f"allergy_randomdietary_preferences_random_number_number: {dietary_preferences_random_number}")
  print (f"customer_age: {customer_age}")
  print (f"job_profile: {job_profile}")

  prompt = f"""You are an inventive storyteller, able to weave captivating narratives from data. Imagine you're writing a character sketch for a "Chocolate A.I." coffee enthusiast, breathing life into their profile.

 Here's the basic framework:
 Customer ID: {customer_id}
 Customer Name: {customer_name}
 Customer Age: {customer_age}
 Customer Inception Date: {customer_inception_date}
 Customer Country Code: {country_code}
 Customer City: {city_name}
 Customer Job Profile: {job_profile}

 Now, let your imagination run wild! Fill in the following, making them unexpected yet plausible:

 Occupation: Anything from an enigmatic "none" to a passionate student or a seasoned professional in any field.  It should relate to their job profile of "{job_profile}".
 Coffee Preferences: The coffee they savor, the way they brew it, the time of day they indulge. Think beyond the ordinary - perhaps they're a connoisseur of rare beans or a devotee of quirky brewing methods.
 Education: This is the highest level of education they've attained. It's a testament to their dedication to learning and their ability to excel.  It should relate to their job profile of "{job_profile}".
 Marital Status: One of these: Single, Married, Divorced, Widowed, Separated, Civil union, Domestic partnership, Common-law marriage.
 Interests: Hobbies and passions that reveal their personality. Let their occupation, education, and marital status guide you, but surprise us with unexpected twists.
 Lifestyle: Paint a picture of their life with a few vivid words. Are they a jet-setting nomad, a quiet bookworm, or a social butterfly?
 Dietary Preferences: Any food allergies they might have, adding a touch of realism to their profile.
 Content Interaction: Engagement with social media posts, email clicks, video views (shows content preferences and brand affinity).
 Customer Service Interactions: Contact reasons, sentiment analysis of interactions, resolution time (highlights pain points and customer satisfaction levels).
 Sports: Any sports  they've played in or sporting teams they like to follow.  Pick sports that make sense based upon the city: {city_name}.
 Solicated Buying Habits: When do they buy coffee? Do they buy it for special occassions, everyday, when meeting friends, or on vacation?
 Children: Do they have kids? Yes or No.  If you any details would be great (age or gender).
 Twitter Handle: Generate a Twitter handle that is unique to this customer.
 Twitter Bio: A concise and engaging bio text (maximum character limit of 250) that reflects the user's interests and passions.
 Twitter Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.
 LinkedIn Handle:  Generate a LinkedIn handle that is unique to this customer.
 LinkedIn Bio: A concise and engaging bio text (maximum character limit of 1000) that reflects the user's interests and passions.
 LinkedIn Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.
 Facebook Handle: Generate a Facebook handle that is unique to this customer.
 Facebook Bio: A concise and engaging bio text (maximum character limit of 1000) that reflects the user's interests and passions.
 Facebook Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.
 Instagram Handle: Generate a Instagram handle that is unique to this customer.
 Instagram Bio: A concise and engaging bio text (maximum character limit of 500) that reflects the user's interests and passions.
 Instagram Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.
 TikTok Handle: Generate a TikTok handle that is unique to this customer.
 TikTok Bio: A concise and engaging bio text (maximum character limit of 250) that reflects the user's interests and passions.
 TikTok Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.
 YouTube Handle: Generate a YouTube handle that is unique to this customer.
 YouTube Bio: A concise and engaging bio text (maximum character limit of 500) that reflects the user's interests and passions.
 YouTube Engagement: (e.g., "Active," "Passive," "Rarely Uses") Determines if they're swayed by coffee drinks or online deals.

 Remember, every customer is unique. Craft profiles that are rich in detail, hinting at hidden depths and untold stories. Let's make these "Chocolate A.I." customers come alive!
 Think outside the box.

 Other instructions:
 Use can leave arrays empty instead of "null".
 Use can leave strings empty instead of "null".
 Use can leave numbers 0 instead of "null".
 Only generate Dietary Preferences if this number is less than 10: {dietary_preferences_random_number}
 There is no need to populate all the Twitter, LinkedIn, Facebook, Instagram, TikTok, and YouTube fields.
 Realistically we might only have a few of the social media data.
 The social media data does not have to pertain to coffee, it can be anything.  Craft a profile rich in detail and let the imagination run wild.
 Base the social media data on the customer ({customer_age}) and the city ({city_name}).
 If we have a handle for a social media account we should then have the bio data.
   - Example: If we have a handle for a Twitter account, we should have the bio data.
   - Example: If we have a handle for a LinkedIn account, we should have the bio data.
 If we have a handle for a social media account we should then have the engagement data.
   - Example: If we have a handle for a Twitter account, we should have the engagement data.
   - Example: If we have a handle for a LinkedIn account, we should have the engagement data.
  """

  # Python (not REST API):         https://ai.google.dev/gemini-api/docs/json-mode?lang=python
  # REST API (no pip installs):    https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output#generativeaionvertexai_gemini_controlled_generation_response_schema-drest
  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
  # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/GenerationConfig
  # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/Schema
  # NOTE: No need to type this by hand, use gemini and prompt it.
  # PROMPT: Write me the json in OpenAPI 3.0 schema object for this object:
  """
  Write me the json in  OpenAPI 3.0 schema object for the below object.
  Make all fields required.
  {
    "customer_id" : 1,
    "customer_age" : 2,
    "occupation" : "string",
    "coffee_preferences" : ["string"],
    "education" : "string",
    "martial_status" : "string",
    "interests" : ["string"],
    "lifestyle" : ["string"],
    "dietary_preferences" : ["string"],
    "content_interaction" : ["string"],
    "customer_service_interactions": [
      {
      "contact_reason": "Order status inquiry",
      "sentiment_analysis": "Neutral",
      "resolution_time": "hours"
      }
    ],
    "sports_teams": ["string"],
    "solicated_buying_habits": ["string"],
    "children": "string",
    "twitter_handle": "string",
    "twitter_bio": "string",
    "twitter_engagement": "string",
    "linkedin_handle": "string",
    "linkedin_bio": "string",
    "linkedin_engagement": "string",
    "facebook_handle": "string",
    "facebook_bio": "string",
    "facebook_engagement": "string",
    "instagram_handle": "string",
    "instagram_bio": "string",
    "instagram_engagement": "string",
    "tiktok_handle": "string",
    "tiktok_bio": "string",
    "tiktok_engagement": "string",
    "youtube_handle": "string",
    "youtube_bio": "string",
    "youtube_engagement": "string"
  }
  """
  response_schema = {
    "type": "object",
    "properties": {
      "customer_id": {
        "type": "integer",
        "description": "Unique identifier for the customer"
      },
      "customer_age": {
        "type": "integer",
        "description": "Age of the customer"
      },
      "occupation": {
        "type": "string",
        "description": "Customer's occupation"
      },
      "coffee_preferences": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Coffee preference (e.g., type, brewing method, time of day)"
        },
        "description": "List of customer's coffee preferences"
      },
      "education": {
        "type": "string",
        "description": "Customer's highest level of education (e.g., degree)"
      },
      "martial_status": {
        "type": "string",
        "description": "Customer's marital status (e.g., married, single)"
      },
      "interests": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's interest (e.g., hobbies, activities)"
        },
        "description": "List of customer's interests"
      },
      "lifestyle": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's lifestyle category (e.g., profession, travel habits)"
        },
        "description": "List of customer's lifestyle categories"
      },
      "dietary_preferences": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's dietary preferences (e.g., food allergies)"
        },
        "description": "List of customer's dietary preferences"
      },
      "content_interaction": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's content interaction details (e.g., engagement, email behavior, video viewing)"
        },
        "description": "Details about customer's content interaction"
      },
      "customer_service_interactions": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "contact_reason": {
              "type": "string",
              "description": "Reason for contacting customer service"
            },
            "sentiment_analysis": {
              "type": "string",
              "description": "Sentiment analysis of the customer service interaction (e.g., positive, neutral, negative)"
            },
            "resolution_time": {
              "type": "string",
              "description": "Time taken to resolve the customer service interaction"
            }
          },
          "required": [
            "contact_reason",
            "sentiment_analysis",
            "resolution_time"
          ]
        },
        "description": "Customer's past interactions with customer service"
      },
      "sports": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's favorite sports teams"
        },
        "description": "Details about what sports they follow or like."
      },
      "solicated_buying_habits": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "Customer's favorite reasons for purchaing coffee"
        },
        "description": "Details about when they like to buy coffee"
      },
      "children": {
        "type": "string",
        "description": "Does the customer have children?"
      },
      "twitter_handle": {
        "type": "string",
        "description": "Customer's Twitter handle"
      },
      "twitter_bio": {
        "type": "string",
        "description": "A generated Twitter bio"
      },
      "twitter_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      },
      "linkedin_handle": {
        "type": "string",
        "description": "Customer's LinkedIn handle"
      },
      "linkedin_bio": {
        "type": "string",
        "description": "A generated LinkedIn bio"
      },
      "linkedin_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      },
      "facebook_handle": {
        "type": "string",
        "description": "Customer's Facebook handle"
      },
      "facebook_bio": {
        "type": "string",
        "description": "A generated Facebook bio"
      },
      "facebook_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      },
      "instagram_handle": {
        "type": "string",
        "description": "Customer's Instagram handle"
      },
      "instagram_bio": {
        "type": "string",
        "description": "A generated Instagram bio"
      },
      "instagram_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      },
      "tiktok_handle": {
        "type": "string",
        "description": "Customer's TikTok handle"
      },
      "tiktok_bio": {
        "type": "string",
        "description": "A generated TikTok bio"
      },
      "tiktok_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      },
      "youtube_handle": {
        "type": "string",
        "description": "Customer's YouTube handle"
      },
      "youtube_bio": {
        "type": "string",
        "description": "A generated YouTube bio"
      },
      "youtube_engagement": {
        "type": "string",
        "description": "Active, Passive, or Rarely Uses"
      }
    },
    "required": [
      "customer_id",
      "customer_age",
      "occupation",
      "coffee_preferences",
      "education",
      "martial_status",
      "interests",
      "lifestyle",
      "dietary_preferences",
      "content_interaction",
      "customer_service_interactions",
      "sports",
      "solicated_buying_habits",
      "children",
      "twitter_handle",
      "twitter_bio",
      "twitter_engagement",
      "linkedin_handle",
      "linkedin_bio",
      "facebook_handle",
      "facebook_bio",
      "instagram_handle",
      "instagram_bio",
      "tiktok_handle",
      "tiktok_bio",
      "youtube_handle",
      "youtube_bio"
    ]
  }

  result = GeminiLLM(prompt, response_schema=response_schema, temperature=1)
  return result
  #print()
  #PrettyPrintJson(result)
  #print()


#### Run Generator Function

In [ ]:
# Generate customer data.  Change the LIMIT clause to process more data
sql = """
WITH customer_by_country AS
(
  SELECT order_tbl.customer_id, city.city_name, COUNT(*) AS city_order_cnt
    FROM `chocolate_ai.order` AS order_tbl
         INNER JOIN `chocolate_ai.location` AS location
                 ON order_tbl.location_id = location.location_id
         INNER JOIN `chocolate_ai.city` AS city
                 ON location.city_id = city.city_id
          INNER JOIN `chocolate_ai.customer` AS customer
                 ON order_tbl.customer_id = customer.customer_id
                AND customer.country_code = city.country_code
                AND customer.country_code = 'GBR' -- REMOVE THIS FOR ALL CUSTOMERS
   WHERE order_tbl.customer_id NOT IN (SELECT customer_id FROM `chocolate_ai.customer_marketing_profile`)
   GROUP BY ALL
)
, customer_by_country_ranking AS
(
  SELECT customer_id, city_name, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY city_order_cnt DESC) AS ranking
    FROM customer_by_country
)
SELECT customer.customer_id, customer.customer_name, customer.customer_yob,
       customer.customer_inception_date, customer.country_code, customer_by_country_ranking.city_name
  FROM `chocolate_ai.customer` AS customer
       INNER JOIN customer_by_country_ranking
               ON customer.customer_id = customer_by_country_ranking.customer_id
              AND customer_by_country_ranking.ranking = 1
ORDER BY customer.customer_id
LIMIT 500
"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_id = row.customer_id
  customer_name = row.customer_name
  customer_yob = row.customer_yob
  customer_inception_date = row.customer_inception_date
  country_code = row.country_code
  city_name = row.city_name

  retry = 0
  success = False
  while not success:
    try:
      result = GenerateCustomerMarketingProfile(customer_id, customer_name, customer_yob, customer_inception_date, country_code, city_name)
      PrettyPrintJson(result)
      print()

      # Save to database
      try:
        sql=f"""INSERT INTO `chocolate_ai.customer_marketing_profile`
                            (customer_id, customer_profile_data)
                    VALUES({customer_id}, JSON'{result}')""".replace("'","\'")

        RunQuery(sql)

        # Jump out of loop
        success = True

      except Exception as e:
        retry += 1
        print("---------------------------------------------------------------------------------------")
        print(f"Error executing SQL statement for customer {customer_id}: {sql}")
        print("---------------------------------------------------------------------------------------")

        if retry > 5:
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          print(f"Error executing SQL statement for customer {customer_id} after 5 retries")
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          break # Skip this customer

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Error generating customer marketing profile for customer {customer_id}: {e}")
      print("---------------------------------------------------------------------------------------")

      if retry > 5:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"Error generating customer marketing profile for customer {customer_id} after 5 retries")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        break # Skip this customer


### Step 3: Derive data_beans_profile_data JSON

#### Define Individual SQL Queries

##### Customer lifetime spend and Last Order

```sql
# For each customer in the `customer` table, get the total amount spent by the customer.
# The amount a customer has spent is the sum of `item_total` in the `order_item` table.
# Find which order_item rows apply to each customer by joining the following tables: `customer`, `order`, `order_item`.
SELECT
      c.customer_id,
      SUM(oi.item_total) / 5 AS total_amount_spent, -- Adjusted for the / 5 factor
      AVG(oi.item_total) AS average_amount_spent_per_order,
      COUNT(DISTINCT o.order_id) AS total_orders,
      MAX(order_datetime) AS last_order_date
    FROM
      `chocolate_ai.customer` AS c
      INNER JOIN `chocolate_ai.order` AS o ON c.customer_id = o.customer_id
      INNER JOIN `chocolate_ai.order_item` AS oi ON o.order_id = oi.order_id
    WHERE o.order_datetime  < '2024-09-16'
    GROUP BY 1
  ```

##### Average per order

```sql
# For each customer in the `customer` table, get the average amount spent by the customer per order.
# The amount a customer has spent is the sum of `item_total` in the `order_item` table.
# Find which order_item rows apply to each customer by joining the following tables: `customer`, `order`, `order_item`.
SELECT
    c.customer_id,
    avg(oi.item_total) AS average_amount_spent_per_order
  FROM
    `chocolate_ai.customer` AS c
    INNER JOIN `chocolate_ai.order` AS o ON c.customer_id = o.customer_id
    INNER JOIN `chocolate_ai.order_item` AS oi ON o.order_id = oi.order_id
  GROUP BY 1;
```

##### Total number of orders

```sql
# For each customer in the `customer` table, get the total number of orders.
# Find which orders apply to each customer by joining the following tables: `customer`, `order`.
SELECT
  c.customer_id,
  COUNT(o.order_id) AS total_orders
FROM
  `chocolate_ai.customer` AS c
  INNER JOIN `chocolate_ai.order` AS o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
ORDER BY total_orders DESC;
```

##### Purchase Locations

```sql
# For each customer in the `customer` table, get the list of locations where they have purchased.
# Output the purchase locations as an array.
# The location of a purchase is listed as `location_id` in the `order` table.
# Orders are associated with customers via the `customer_id` column in the `order` table.
SELECT
    t1.customer_id,
    array_agg(DISTINCT t2.location_id) AS purchase_locations
  FROM
    `chocolate_ai.customer` AS t1
    INNER JOIN `chocolate_ai.order` AS t2 ON t1.customer_id = t2.customer_id
  GROUP BY 1;
```

##### Most frequent purchase location

```sql
# For each customer in the `customer` table, get the location where they most frequently purchase.
# Output the purchase locations as an array.
# The location of a purchase is listed as `location_id` in the `order` table.
# Orders are associated with customers via the `customer_id` column in the `order` table.
SELECT
    t1.customer_id,
    ARRAY_AGG(location_id ORDER BY cnt DESC LIMIT 1) AS most_frequent_purchase_location
  FROM
    `chocolate_ai.customer` AS t1
    INNER JOIN (
      SELECT
          customer_id,
          location_id,
          count(*) AS cnt
        FROM
          `chocolate_ai.order`
        GROUP BY 1, 2
    ) AS t2 ON t1.customer_id = t2.customer_id
  GROUP BY 1;
```

##### Favorite Menu Items

```sql
# For each customer in the `customer` table, get the top 3 most commonly purchased items.
# The items a customer has purchased is shown as `menu_id` in the `order_item` table.
# Find which order_item rows apply to each customer by joining the following tables: `customer`, `order`, `order_item`.
# Output the top 3 items as an ordered array per customer
WITH CustomerOrderItemCounts AS (
    SELECT
        c.customer_id,
        oi.menu_id,
        COUNT(*) AS item_count
    FROM
        `chocolate_ai.customer` AS c
    INNER JOIN
        `chocolate_ai.order` AS o
        ON c.customer_id = o.customer_id
    INNER JOIN
        `chocolate_ai.order_item` AS oi
        ON o.order_id = oi.order_id
    GROUP BY
        c.customer_id, oi.menu_id
),
RankedCustomerOrderItemCounts AS (
    SELECT
        customer_id,
        menu_id,
        item_count,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY item_count DESC) AS rn
    FROM
        `CustomerOrderItemCounts`
)
SELECT
    customer_id,
    ARRAY_AGG(menu_id ORDER BY rn) AS top_3_favorite_menu_items
FROM
    `RankedCustomerOrderItemCounts`
WHERE
    rn <= 3
GROUP BY
    customer_id;
```

##### Review counts and sentiment

```sql
# For each customer in the `customer` table, get the number of reviews in the `customer_review` table, the sentiment of their most recent review, the percentage of their positive reviews, the percentage of their negative reviews, and the percentage of their netural reviews.  Sentiment is defined by `review_sentiment` and the most common sentiment of their reviews.
WITH CustomerReviewCounts AS (
    SELECT
        customer_id,
        COUNT(customer_review_id) AS total_reviews,
        COUNTIF(review_sentiment = 'Positive') AS positive_reviews,
        COUNTIF(review_sentiment = 'Negative') AS negative_reviews,
        COUNTIF(review_sentiment = 'Neutral') AS neutral_reviews
    FROM
        `chocolate_ai.customer_review`
    GROUP BY
        customer_id
),

CustomerLatestReview AS (
  SELECT
    customer_id,
    ARRAY_AGG(review_sentiment ORDER BY review_datetime DESC LIMIT 1)[SAFE_OFFSET(0)] AS latest_review_sentiment
  FROM
    `chocolate_ai.customer_review`
  GROUP BY
    customer_id
)

SELECT
    c.customer_id,
    COALESCE(crc.total_reviews, 0) AS total_reviews,
    clr.latest_review_sentiment,
    SAFE_DIVIDE(crc.positive_reviews * 100, crc.total_reviews) AS positive_review_percentage,
    SAFE_DIVIDE(crc.negative_reviews * 100, crc.total_reviews) AS negative_review_percentage,
    SAFE_DIVIDE(crc.neutral_reviews * 100, crc.total_reviews) AS neutral_review_percentage
FROM
    `chocolate_ai.customer` AS c
LEFT JOIN
    `CustomerReviewCounts` AS crc ON c.customer_id = crc.customer_id
LEFT JOIN
  `CustomerLatestReview` AS clr ON c.customer_id = clr.customer_id
ORDER BY
    c.customer_id;
```


#### Define Combined Query



```sql
# Combine the BigQuery queries above into a single query and output as a single json object per customer_id.

WITH
  customer_lifetime_spend AS (
    SELECT
      c.customer_id,
      SUM(oi.item_total) / 5 AS total_amount_spent, -- Adjusted for the / 5 factor
      AVG(oi.item_total) AS average_amount_spent_per_order,
      COUNT(DISTINCT o.order_id) AS total_orders,
      MAX(order_datetime) AS last_order_date
    FROM
      `chocolate_ai.customer` AS c
      INNER JOIN `chocolate_ai.order` AS o ON c.customer_id = o.customer_id
      INNER JOIN `chocolate_ai.order_item` AS oi ON o.order_id = oi.order_id
    WHERE o.order_datetime  < '2024-09-16'
    GROUP BY 1
  ),

  purchase_locations AS (
    SELECT
      t1.customer_id,
      ARRAY_AGG(DISTINCT t2.location_id) AS purchase_locations,
      ARRAY_AGG(location_id ORDER BY cnt DESC LIMIT 1)[SAFE_OFFSET(0)] AS most_frequent_purchase_location
    FROM
      `chocolate_ai.customer` AS t1
      INNER JOIN (
        SELECT
          customer_id,
          location_id,
          COUNT(*) AS cnt
        FROM
          `chocolate_ai.order`
        GROUP BY 1, 2
      ) AS t2 ON t1.customer_id = t2.customer_id
    GROUP BY 1
  ),

  favorite_menu_items AS (
    WITH CustomerOrderItemCounts AS (
      SELECT
        c.customer_id,
        oi.menu_id,
        COUNT(*) AS item_count
      FROM
        `chocolate_ai.customer` AS c
      INNER JOIN
        `chocolate_ai.order` AS o
        ON c.customer_id = o.customer_id
      INNER JOIN
        `chocolate_ai.order_item` AS oi
        ON o.order_id = oi.order_id
      GROUP BY
        c.customer_id, oi.menu_id
    ),
    RankedCustomerOrderItemCounts AS (
      SELECT
        customer_id,
        menu_id,
        item_count,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY item_count DESC) AS rn
      FROM
        `CustomerOrderItemCounts`
    )
    SELECT
      customer_id,
      ARRAY_AGG(menu_id ORDER BY rn) AS top_3_favorite_menu_items
    FROM
      `RankedCustomerOrderItemCounts`
    WHERE
      rn <= 3
    GROUP BY
      customer_id
  ),

  review_data AS (
    WITH CustomerReviewCounts AS (
      SELECT
          customer_id,
          COUNT(customer_review_id) AS total_reviews,
          COUNTIF(review_sentiment = 'Positive') AS positive_reviews,
          COUNTIF(review_sentiment = 'Negative') AS negative_reviews,
          COUNTIF(review_sentiment = 'Neutral') AS neutral_reviews
      FROM
          `chocolate_ai.customer_review`
      GROUP BY
          customer_id
    ),

    CustomerLatestReview AS (
      SELECT
        customer_id,
        ARRAY_AGG(review_sentiment ORDER BY review_datetime DESC LIMIT 1)[SAFE_OFFSET(0)] AS latest_review_sentiment
      FROM
        `chocolate_ai.customer_review`
      GROUP BY
        customer_id
    )

    SELECT
        c.customer_id,
        COALESCE(crc.total_reviews, 0) AS total_reviews,
        clr.latest_review_sentiment,
        SAFE_DIVIDE(crc.positive_reviews * 100, crc.total_reviews) AS positive_review_percentage,
        SAFE_DIVIDE(crc.negative_reviews * 100, crc.total_reviews) AS negative_review_percentage,
        SAFE_DIVIDE(crc.neutral_reviews * 100, crc.total_reviews) AS neutral_review_percentage
    FROM
        `chocolate_ai.customer` AS c
    LEFT JOIN
        `CustomerReviewCounts` AS crc ON c.customer_id = crc.customer_id
    LEFT JOIN
      `CustomerLatestReview` AS clr ON c.customer_id = clr.customer_id
    ORDER BY
        c.customer_id
  )

SELECT
  clv.customer_id,
  TO_JSON_STRING(STRUCT(
    clv.total_amount_spent,
    clv.average_amount_spent_per_order,
    clv.total_orders,
    clv.last_order_date,
    pl.purchase_locations,
    pl.most_frequent_purchase_location,
    fmi.top_3_favorite_menu_items,
    rd.total_reviews,
    rd.latest_review_sentiment,
    rd.positive_review_percentage,
    rd.negative_review_percentage,
    rd.neutral_review_percentage
  )) AS customer_data
FROM
  customer_lifetime_spend AS clv
  JOIN purchase_locations AS pl ON clv.customer_id = pl.customer_id
  JOIN review_data AS rd ON clv.customer_id = rd.customer_id
  JOIN favorite_menu_items AS fmi ON clv.customer_id = fmi.customer_id;
```

#### Run Combined Query and Update customer_marketing_profile

In [ ]:
sql=f"""UPDATE `chocolate_ai.customer_marketing_profile` AS mp
SET data_beans_profile_data = PARSE_JSON(data_beans_profile.customer_data, wide_number_mode=>'round')
FROM (
  WITH
      customer_lifetime_spend AS (
        SELECT
          c.customer_id,
          SUM(oi.item_total) / 5 AS total_amount_spent,
          AVG(oi.item_total) AS average_amount_spent_per_order,
          COUNT(DISTINCT o.order_id) AS total_orders,
          MAX(order_datetime) AS last_order_date
        FROM
          `chocolate_ai.customer` AS c
          INNER JOIN `chocolate_ai.order` AS o ON c.customer_id = o.customer_id
          INNER JOIN `chocolate_ai.order_item` AS oi ON o.order_id = oi.order_id
        WHERE o.order_datetime  < '2024-09-16'
        GROUP BY 1
      ),

      purchase_locations AS (
        SELECT
          t1.customer_id,
          ARRAY_AGG(DISTINCT t2.location_id) AS purchase_locations,
          ARRAY_AGG(location_id ORDER BY cnt DESC LIMIT 1)[SAFE_OFFSET(0)] AS most_frequent_purchase_location
        FROM
          `chocolate_ai.customer` AS t1
          INNER JOIN (
            SELECT
              customer_id,
              location_id,
              COUNT(*) AS cnt
            FROM
              `chocolate_ai.order`
            GROUP BY 1, 2
          ) AS t2 ON t1.customer_id = t2.customer_id
        GROUP BY 1
      ),

      favorite_menu_items AS (
        WITH CustomerOrderItemCounts AS (
          SELECT
            c.customer_id,
            oi.menu_id,
            COUNT(*) AS item_count
          FROM
            `chocolate_ai.customer` AS c
          INNER JOIN
            `chocolate_ai.order` AS o
            ON c.customer_id = o.customer_id
          INNER JOIN
            `chocolate_ai.order_item` AS oi
            ON o.order_id = oi.order_id
          GROUP BY
            c.customer_id, oi.menu_id
        ),
        RankedCustomerOrderItemCounts AS (
          SELECT
            customer_id,
            menu_id,
            item_count,
            ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY item_count DESC) AS rn
          FROM
            `CustomerOrderItemCounts`
        )
        SELECT
          customer_id,
          ARRAY_AGG(menu_id ORDER BY rn) AS top_3_favorite_menu_items
        FROM
          `RankedCustomerOrderItemCounts`
        WHERE
          rn <= 3
        GROUP BY
          customer_id
      ),

      review_data AS (
        WITH CustomerReviewCounts AS (
          SELECT
              customer_id,
              COUNT(customer_review_id) AS total_reviews,
              COUNTIF(review_sentiment = 'Positive') AS positive_reviews,
              COUNTIF(review_sentiment = 'Negative') AS negative_reviews,
              COUNTIF(review_sentiment = 'Neutral') AS neutral_reviews
          FROM
              `chocolate_ai.customer_review`
          GROUP BY
              customer_id
        ),

        CustomerLatestReview AS (
          SELECT
            customer_id,
            ARRAY_AGG(review_sentiment ORDER BY review_datetime DESC LIMIT 1)[SAFE_OFFSET(0)] AS latest_review_sentiment
          FROM
            `chocolate_ai.customer_review`
          GROUP BY
            customer_id
        )

        SELECT
            c.customer_id,
            COALESCE(crc.total_reviews, 0) AS total_reviews,
            clr.latest_review_sentiment,
            SAFE_DIVIDE(crc.positive_reviews * 100, crc.total_reviews) AS positive_review_percentage,
            SAFE_DIVIDE(crc.negative_reviews * 100, crc.total_reviews) AS negative_review_percentage,
            SAFE_DIVIDE(crc.neutral_reviews * 100, crc.total_reviews) AS neutral_review_percentage
        FROM
            `chocolate_ai.customer` AS c
        LEFT JOIN
            `CustomerReviewCounts` AS crc ON c.customer_id = crc.customer_id
        LEFT JOIN
          `CustomerLatestReview` AS clr ON c.customer_id = clr.customer_id
        ORDER BY
            c.customer_id
      ),

      profile_cte AS (
        SELECT
          clv.customer_id,
          TO_JSON_STRING(STRUCT(
            clv.total_amount_spent,
            clv.average_amount_spent_per_order,
            clv.total_orders,
            clv.last_order_date,
            pl.purchase_locations,
            pl.most_frequent_purchase_location,
            fmi.top_3_favorite_menu_items,
            rd.total_reviews,
            rd.latest_review_sentiment,
            rd.positive_review_percentage,
            rd.negative_review_percentage,
            rd.neutral_review_percentage
          )) AS customer_data
        FROM
          customer_lifetime_spend AS clv
          JOIN purchase_locations AS pl ON clv.customer_id = pl.customer_id
          JOIN review_data AS rd ON clv.customer_id = rd.customer_id
          JOIN favorite_menu_items AS fmi ON clv.customer_id = fmi.customer_id
    )
  SELECT customer_id, customer_data FROM profile_cte
) AS data_beans_profile
WHERE mp.customer_id = data_beans_profile.customer_id;
"""

RunQuery(sql)

### Step 4: Generate generated_marketing_insights JSON

#### Customer Segmentation

**Prompts**

The following prompts were used in multi-turn chat with Gemini to generate the recommended customer segmentation JSON schema:
- What are some typical customer segments that marketing teams care about?
- Generate a JSON document of all of these customer segments and subsegments.
- Add 3 more plausible subsegments to each segment.
- For each subsegment, generate an array of up to 10 possible categorical values that could be assigned to a customer for that segment and subsegment. If the categorical value is numerical, use range binning instead of discrete values.
- Update subsegments "Specific Needs," "Pain Points," "Challenges," "Goals," "Priorities," "New Leads," "Potential Customers," "First-Time Customers," "Repeat Customers," "Loyal Advocates," "At-Risk Customers," "Former Customers," and "Inactive Customers" for the product offering of "Mobile Coffee Shop products".
- Generate a prompt to instruct an LLM to review two JSON objects with information about a customer (customer_profile_data and data_beans_profile_data) and then assign segments to the customer based on the JSON schema you just helped me generate.


**Resulting JSON Schema**

```json
{
      "customer_segments": {
          "demographic_segmentation": {
              "subsegments": {
                  "Age": ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"],
                  "Gender": ["Male", "Female", "Non-binary", "Prefer not to say"],
                  "Income": ["<$25k", "$25k-$50k", "$50k-$75k", "$75k-$100k", "$100k-$150k", "$150k+"],
                  "Education": ["Less than High School", "High School Graduate", "Some College", "Associate's Degree", "Bachelor's Degree", "Master's Degree", "Doctorate"],
                  "Marital Status": ["Single", "Married", "Divorced", "Widowed", "Domestic Partnership"],
                  "Occupation": ["Professional", "Managerial", "Technical", "Sales", "Service", "Administrative", "Production", "Retired", "Student", "Unemployed"],
                  "Family Size": ["1", "2", "3", "4", "5", "6+"],
                  "Ethnicity": ["White", "Black or African American", "Hispanic or Latino", "Asian", "Native American or Alaska Native", "Native Hawaiian or Other Pacific Islander", "Two or More Races"],
                  "Generation": ["Baby Boomer", "Generation X", "Millennial", "Generation Z"],
                  "Language": ["English", "Spanish", "Chinese", "French", "German", "Arabic", "Hindi", "Portuguese", "Russian", "Japanese"]
              }
          },
          "geographic_segmentation": {
              "subsegments": {
                  "Country": ["United States", "Canada", "Mexico", "United Kingdom", "Germany", "France", "China", "India", "Brazil", "Japan"],
                  "Region": ["Northeast", "Midwest", "South", "West"],
                  "City": ["New York", "Los Angeles", "Chicago", "Houston", "Philadelphia", "Phoenix", "San Antonio", "San Diego", "Dallas", "San Jose"],
                  "Climate": ["Tropical", "Subtropical", "Temperate", "Continental", "Polar"],
                  "Population Density": ["High", "Medium", "Low"],
                  "Urban/Rural": ["Urban", "Suburban", "Rural"],
                  "Time Zone": ["EST", "CST", "MST", "PST"]
              }
          },
          "psychographic_segmentation": {
              "subsegments": {
                  "Personality": ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"],
                  "Lifestyle": ["Active", "Sedentary", "Family-oriented", "Career-driven", "Social"],
                  "Interests": ["Sports", "Travel", "Fashion", "Technology", "Food", "Music", "Art", "Reading", "Gaming", "Outdoor Activities"],
                  "Values": ["Achievement", "Security", "Self-Direction", "Stimulation", "Hedonism", "Power", "Tradition", "Conformity", "Benevolence", "Universalism"],
                  "Attitudes": ["Positive", "Negative", "Neutral"],
                  "Hobbies": ["Cooking", "Gardening", "Photography", "DIY", "Collecting", "Volunteering", "Writing", "Music", "Sports", "Gaming"],
                  "Social Class": ["Upper Class", "Upper Middle Class", "Lower Middle Class", "Working Class", "Lower Class"],
                  "Motivations": ["Intrinsic", "Extrinsic"]
              }
          },
          "behavioral_segmentation": {
              "subsegments": {
                  "Purchase History": ["High Spender", "Medium Spender", "Low Spender", "Frequent Buyer", "Occasional Buyer", "New Buyer", "Repeat Buyer", "Loyal Customer", "Price Sensitive", "Brand Loyal"],
                  "Browsing Behavior": ["Long Sessions", "Short Sessions", "Frequent Visits", "Occasional Visits", "Mobile User", "Desktop User", "High Engagement", "Low Engagement", "Product Focused", "Content Focused"],
                  "Loyalty Status": ["Loyal", "Repeat", "New", "At-Risk", "Inactive"],
                  "Usage Frequency": ["Heavy User", "Moderate User", "Light User"],
                  "Benefits Sought": ["Quality", "Price", "Convenience", "Service", "Style", "Status", "Innovation", "Sustainability", "Experience", "Community"],
                  "Occasion/Timing": ["Holiday", "Birthday", "Anniversary", "Seasonal", "Weekend", "Weekday", "Morning", "Afternoon", "Evening"],
                  "User Status": ["Active", "Inactive", "Lapsed", "Potential", "New"],
                  "Spending Habits": ["High", "Medium", "Low"]
              }
          },
          "technographic_segmentation": {
              "subsegments": {
                  "Devices": ["Smartphone", "Tablet", "Laptop", "Desktop", "Smart TV", "Wearable", "Gaming Console", "Smart Home Device"],
                  "Operating Systems": ["iOS", "Android", "Windows", "macOS", "Linux"],
                  "Browsers": ["Chrome", "Safari", "Firefox", "Edge", "Opera"],
                  "Software": ["Microsoft Office", "Adobe Creative Cloud", "Antivirus", "Productivity Apps", "Gaming Software"],
                  "Social Media Platforms": ["Facebook", "Instagram", "Twitter", "LinkedIn", "TikTok", "Snapchat", "Pinterest", "YouTube"],
                  "Internet Connectivity": ["Broadband", "Mobile", "Dial-up", "Satellite"],
                  "Tech Savviness": ["Early Adopter", "Mainstream", "Laggard"],
                  "Adoption Rate": ["High", "Medium", "Low"]
              }
          },
          "needs_based_segmentation": {
              "subsegments": {
                  "Specific Needs": ["Quick & Convenient Coffee", "Variety of Coffee Options", "Freshly Brewed Coffee", "Specialty Coffee Drinks", "Pastries & Snacks", "Mobile Ordering & Payment", "Loyalty Program", "Outdoor Seating", "Catering Services"],
                  "Pain Points": ["Long Lines at Traditional Coffee Shops", "Limited Coffee Options", "Inconsistent Coffee Quality", "High Prices", "Limited Accessibility", "Inconvenient Locations", "Lack of Mobile Ordering", "No Loyalty Program"],
                  "Challenges": ["Finding Quality Coffee on the Go", "Limited Time in the Morning", "Dietary Restrictions", "Budget Constraints", "Lack of Nearby Coffee Options"],
                  "Goals": ["Enjoy Delicious Coffee Anytime, Anywhere", "Save Time & Money", "Discover New Coffee Flavors", "Support Local Businesses", "Socialize & Connect with Others"],
                  "Priorities": ["Convenience", "Quality", "Affordability", "Variety", "Sustainability"]
              }
          },
          "value_based_segmentation": {
              "subsegments": {
                  "Perceived Value": ["High", "Medium", "Low"],
                  "Price Sensitivity": ["High", "Medium", "Low"],
                  "Willingness to Pay": ["High", "Medium", "Low"],
                  "Cost-Benefit Analysis": ["Value-Driven", "Price-Driven"]
              }
          },
          "customer_lifecycle_segmentation": {
              "subsegments": {
                  "New Leads": ["Subscribed to Newsletter", "Followed on Social Media", "Visited Website", "Downloaded App"],
                  "Potential Customers": ["Inquired About Services", "Requested a Quote", "Visited Mobile Shop Location"],
                  "First-Time Customers": ["Placed First Order", "Tried One Product"],
                  "Repeat Customers": ["Placed Multiple Orders", "Tried Multiple Products"],
                  "Loyal Advocates": ["Regularly Purchases", "Refers Friends & Family", "Leaves Positive Reviews", "Engages on Social Media"],
                  "At-Risk Customers": ["Decreased Purchase Frequency", "Expressed Dissatisfaction", "Unengaged with Brand"],
                  "Former Customers": ["Stopped Purchasing", "Switched to Competitor", "Unsubscribed from Communications"],
                  "Inactive Customers": ["Hasn't Purchased in a While", "Doesn't Engage with Brand"]
              }
          }
      }
  }
```

**Resulting LLM prompt**

You will be given two JSON objects containing customer profile information: `customer_profile_data` and `data_beans_profile_data`. Your task is to analyze this data and assign relevant customer segments to the customer based on the following JSON schema:

```json
{
    "customer_segments": {
        // ... (the JSON schema you provided)
    }
}
```

#### Define Customer Segmentation Function

In [ ]:
def GenerateCustomerSegments(customer_id, customer_profile_data, data_beans_profile_data):
  # For each row in customer_marketing_profile, send this prompt:
  prompt = """
  You will be given two JSON objects containing customer profile information:
  customer_profile_data and data_beans_profile_data. Your task is to analyze
  this data and assign relevant customer segments to the customer based on the
  following JSON schema:

  {
      "customer_segments": {
          "demographic_segmentation": {
              "subsegments": {
                  "Age": ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"],
                  "Gender": ["Male", "Female", "Non-binary", "Prefer not to say"],
                  "Income": ["<$25k", "$25k-$50k", "$50k-$75k", "$75k-$100k", "$100k-$150k", "$150k+"],
                  "Education": ["Less than High School", "High School Graduate", "Some College", "Associate's Degree", "Bachelor's Degree", "Master's Degree", "Doctorate"],
                  "Marital Status": ["Single", "Married", "Divorced", "Widowed", "Domestic Partnership"],
                  "Occupation": ["Professional", "Managerial", "Technical", "Sales", "Service", "Administrative", "Production", "Retired", "Student", "Unemployed"],
                  "Family Size": ["1", "2", "3", "4", "5", "6+"],
                  "Ethnicity": ["White", "Black or African American", "Hispanic or Latino", "Asian", "Native American or Alaska Native", "Native Hawaiian or Other Pacific Islander", "Two or More Races"],
                  "Generation": ["Baby Boomer", "Generation X", "Millennial", "Generation Z"],
                  "Language": ["English", "Spanish", "Chinese", "French", "German", "Arabic", "Hindi", "Portuguese", "Russian", "Japanese"]
              }
          },
          "geographic_segmentation": {
              "subsegments": {
                  "Country": ["United States", "Canada", "Mexico", "United Kingdom", "Germany", "France", "China", "India", "Brazil", "Japan"],
                  "Region": ["Northeast", "Midwest", "South", "West"],
                  "City": ["New York", "Los Angeles", "Chicago", "Houston", "Philadelphia", "Phoenix", "San Antonio", "San Diego", "Dallas", "San Jose"],
                  "Climate": ["Tropical", "Subtropical", "Temperate", "Continental", "Polar"],
                  "Population Density": ["High", "Medium", "Low"],
                  "Urban/Rural": ["Urban", "Suburban", "Rural"],
                  "Time Zone": ["EST", "CST", "MST", "PST"]
              }
          },
          "psychographic_segmentation": {
              "subsegments": {
                  "Personality": ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"],
                  "Lifestyle": ["Active", "Sedentary", "Family-oriented", "Career-driven", "Social"],
                  "Interests": ["Sports", "Travel", "Fashion", "Technology", "Food", "Music", "Art", "Reading", "Gaming", "Outdoor Activities"],
                  "Values": ["Achievement", "Security", "Self-Direction", "Stimulation", "Hedonism", "Power", "Tradition", "Conformity", "Benevolence", "Universalism"],
                  "Attitudes": ["Positive", "Negative", "Neutral"],
                  "Hobbies": ["Cooking", "Gardening", "Photography", "DIY", "Collecting", "Volunteering", "Writing", "Music", "Sports", "Gaming"],
                  "Social Class": ["Upper Class", "Upper Middle Class", "Lower Middle Class", "Working Class", "Lower Class"],
                  "Motivations": ["Intrinsic", "Extrinsic"]
              }
          },
          "behavioral_segmentation": {
              "subsegments": {
                  "Purchase History": ["High Spender", "Medium Spender", "Low Spender", "Frequent Buyer", "Occasional Buyer", "New Buyer", "Repeat Buyer", "Loyal Customer", "Price Sensitive", "Brand Loyal"],
                  "Browsing Behavior": ["Long Sessions", "Short Sessions", "Frequent Visits", "Occasional Visits", "Mobile User", "Desktop User", "High Engagement", "Low Engagement", "Product Focused", "Content Focused"],
                  "Loyalty Status": ["Loyal", "Repeat", "New", "At-Risk", "Inactive"],
                  "Usage Frequency": ["Heavy User", "Moderate User", "Light User"],
                  "Benefits Sought": ["Quality", "Price", "Convenience", "Service", "Style", "Status", "Innovation", "Sustainability", "Experience", "Community"],
                  "Occasion/Timing": ["Holiday", "Birthday", "Anniversary", "Seasonal", "Weekend", "Weekday", "Morning", "Afternoon", "Evening"],
                  "User Status": ["Active", "Inactive", "Lapsed", "Potential", "New"],
                  "Spending Habits": ["High", "Medium", "Low"]
              }
          },
          "technographic_segmentation": {
              "subsegments": {
                  "Devices": ["Smartphone", "Tablet", "Laptop", "Desktop", "Smart TV", "Wearable", "Gaming Console", "Smart Home Device"],
                  "Operating Systems": ["iOS", "Android", "Windows", "macOS", "Linux"],
                  "Browsers": ["Chrome", "Safari", "Firefox", "Edge", "Opera"],
                  "Software": ["Microsoft Office", "Adobe Creative Cloud", "Antivirus", "Productivity Apps", "Gaming Software"],
                  "Social Media Platforms": ["Facebook", "Instagram", "Twitter", "LinkedIn", "TikTok", "Snapchat", "Pinterest", "YouTube"],
                  "Internet Connectivity": ["Broadband", "Mobile", "Dial-up", "Satellite"],
                  "Tech Savviness": ["Early Adopter", "Mainstream", "Laggard"],
                  "Adoption Rate": ["High", "Medium", "Low"]
              }
          },
          "needs_based_segmentation": {
              "subsegments": {
                  "Specific Needs": ["Quick & Convenient Coffee", "Variety of Coffee Options", "Freshly Brewed Coffee", "Specialty Coffee Drinks", "Pastries & Snacks", "Mobile Ordering & Payment", "Loyalty Program", "Outdoor Seating", "Catering Services"],
                  "Pain Points": ["Long Lines at Traditional Coffee Shops", "Limited Coffee Options", "Inconsistent Coffee Quality", "High Prices", "Limited Accessibility", "Inconvenient Locations", "Lack of Mobile Ordering", "No Loyalty Program"],
                  "Challenges": ["Finding Quality Coffee on the Go", "Limited Time in the Morning", "Dietary Restrictions", "Budget Constraints", "Lack of Nearby Coffee Options"],
                  "Goals": ["Enjoy Delicious Coffee Anytime, Anywhere", "Save Time & Money", "Discover New Coffee Flavors", "Support Local Businesses", "Socialize & Connect with Others"],
                  "Priorities": ["Convenience", "Quality", "Affordability", "Variety", "Sustainability"]
              }
          },
          "value_based_segmentation": {
              "subsegments": {
                  "Perceived Value": ["High", "Medium", "Low"],
                  "Price Sensitivity": ["High", "Medium", "Low"],
                  "Willingness to Pay": ["High", "Medium", "Low"],
                  "Cost-Benefit Analysis": ["Value-Driven", "Price-Driven"]
              }
          },
          "customer_lifecycle_segmentation": {
              "subsegments": {
                  "New Leads": ["Subscribed to Newsletter", "Followed on Social Media", "Visited Website", "Downloaded App"],
                  "Potential Customers": ["Inquired About Services", "Requested a Quote", "Visited Mobile Shop Location"],
                  "First-Time Customers": ["Placed First Order", "Tried One Product"],
                  "Repeat Customers": ["Placed Multiple Orders", "Tried Multiple Products"],
                  "Loyal Advocates": ["Regularly Purchases", "Refers Friends & Family", "Leaves Positive Reviews", "Engages on Social Media"],
                  "At-Risk Customers": ["Decreased Purchase Frequency", "Expressed Dissatisfaction", "Unengaged with Brand"],
                  "Former Customers": ["Stopped Purchasing", "Switched to Competitor", "Unsubscribed from Communications"],
                  "Inactive Customers": ["Hasn't Purchased in a While", "Doesn't Engage with Brand"]
              }
          }
      }
  }


  """

  prompt = prompt + f"""
  Here is the customer's customer_profile_data:
  {customer_profile_data}

  Here is the customer's data_beans_profile_data:
  {data_beans_profile_data}


  Additional instructions:
  - Do not omit any segments or subsegments from your response.
  - Avoid assigning null values as much as possible.
  - Results can be creative, but they must be plausible.

  Now assign relevant customer segments to the customer. Think step by step and explain your reasoning.

  """

  # Ref: https://cloud.google.com/vertex-ai/docs/reference/rest/v1/Schema
  response_schema = {
    "type": "object",
    "properties": {
      "customer_segments": {
        "type": "object",
        "properties": {
          "demographic_segmentation": {
            "type": "object",
            "properties": {
              "Age": { "type": "string" },
              "Gender": { "type": "string" },
              "Income": { "type": "string" },
              "Education": { "type": "string" },
              "Marital Status": { "type": "string" },
              "Occupation": { "type": "string" },
              "Family Size": { "type": "string" },
              "Ethnicity": { "type": "string" },
              "Generation": { "type": "string" },
              "Language": { "type": "string" }
            },
            "required": ["Age", "Gender", "Income", "Education", "Marital Status", "Occupation", "Family Size", "Ethnicity", "Generation", "Language"]
          },
          "geographic_segmentation": {
            "type": "object",
            "properties": {
              "Country": { "type": "string" },
              "Region": { "type": "string" },
              "City": { "type": "string" },
              "Climate": { "type": "string" },
              "Population Density": { "type": "string" },
              "Urban/Rural": { "type": "string" },
              "Time Zone": { "type": "string" }
            },
            "required": ["Country", "Region", "City", "Climate", "Population Density", "Urban/Rural", "Time Zone"]
          },
          "psychographic_segmentation": {
            "type": "object",
            "properties": {
              "Personality": { "type": "string" },
              "Lifestyle": { "type": "string" },
              "Interests": { "type": "string" },
              "Values": { "type": "string" },
              "Attitudes": { "type": "string" },
              "Hobbies": { "type": "array", "items": { "type": "string" } },
              "Social Class": { "type": "string" },
              "Motivations": { "type": "string" }
            },
            "required": ["Personality", "Lifestyle", "Interests", "Values", "Attitudes", "Hobbies", "Social Class", "Motivations"]
          },
          "behavioral_segmentation": {
            "type": "object",
            "properties": {
              "Purchase History": { "type": "string" },
              "Browsing Behavior": { "type": "string" },
              "Loyalty Status": { "type": "string" },
              "Usage Frequency": { "type": "string" },
              "Benefits Sought": { "type": "string" },
              "Occasion/Timing": { "type": "string" },
              "User Status": { "type": "string" },
              "Spending Habits": { "type": "string" }
            },
            "required": ["Purchase History", "Browsing Behavior", "Loyalty Status", "Usage Frequency", "Benefits Sought", "Occasion/Timing", "User Status", "Spending Habits"]
          },
          "technographic_segmentation": {
            "type": "object",
            "properties": {
              "Devices": { "type": "string" },
              "Operating Systems": { "type": "string" },
              "Browsers": { "type": "string" },
              "Software": { "type": "string" },
              "Social Media Platforms": { "type": "string" },
              "Internet Connectivity": { "type": "string" },
              "Tech Savviness": { "type": "string" },
              "Adoption Rate": { "type": "string" }
            },
            "required": ["Devices", "Operating Systems", "Browsers", "Software", "Social Media Platforms", "Internet Connectivity", "Tech Savviness", "Adoption Rate"]
          },
          "needs_based_segmentation": {
            "type": "object",
            "properties": {
              "Specific Needs": { "type": "string" },
              "Pain Points": { "type": "string" },
              "Challenges": { "type": "string" },
              "Goals": { "type": "string" },
              "Priorities": { "type": "string" }
            },
            "required": ["Specific Needs", "Pain Points", "Challenges", "Goals", "Priorities"]
          },
          "value_based_segmentation": {
            "type": "object",
            "properties": {
              "Perceived Value": { "type": "string" },
              "Price Sensitivity": { "type": "string" },
              "Willingness to Pay": { "type": "string" },
              "Cost-Benefit Analysis": { "type": "string" }
            },
            "required": ["Perceived Value", "Price Sensitivity", "Willingness to Pay", "Cost-Benefit Analysis"]
          },
          "customer_lifecycle_segmentation": {
            "type": "object",
            "properties": {
              "New Leads": { "type": "array", "items": { "type": "string" } },
              "Potential Customers": { "type": "array", "items": { "type": "string" } },
              "First-Time Customers": { "type": "array", "items": { "type": "string" } },
              "Repeat Customers": { "type": "array", "items": { "type": "string" } },
              "Loyal Advocates": { "type": "array", "items": { "type": "string" } },
              "At-Risk Customers": { "type": "array", "items": { "type": "string" } },
              "Former Customers": { "type": "array", "items": { "type": "string" } },
              "Inactive Customers": { "type": "array", "items": { "type": "string" } }
            },
            "required": ["New Leads", "Potential Customers", "First-Time Customers", "Repeat Customers", "Loyal Advocates", "At-Risk Customers", "Former Customers", "Inactive Customers"]
          }
        },
        "required": ["demographic_segmentation", "geographic_segmentation", "psychographic_segmentation", "behavioral_segmentation", "technographic_segmentation", "needs_based_segmentation", "value_based_segmentation", "customer_lifecycle_segmentation"]
      }
    }
  }

  result = GeminiLLM(prompt, response_schema=response_schema)
  return result


#### Run Customer Segmentation Function

In [ ]:
# Process the ones that need to be populated
sql = """
SELECT *
  FROM `chocolate_ai.customer_marketing_profile`
 WHERE customer_profile_data        IS NOT NULL
   AND data_beans_profile_data      IS NOT NULL
   AND generated_marketing_insights IS NULL
"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_id = row.customer_id
  customer_profile_data = row.customer_profile_data
  data_beans_profile_data = row.data_beans_profile_data

  retry = 0
  success = False
  while not success:
    try:
      result = GenerateCustomerSegments(customer_id, customer_profile_data, data_beans_profile_data)
      #PrettyPrintJson(result)
      print()

      # Save to database
      try:
        sql=f"""UPDATE `chocolate_ai.customer_marketing_profile`
                SET generated_marketing_insights = JSON'{result}'
                WHERE customer_id = {customer_id};""".replace("'","\'")

        RunQuery(sql)

        # Jump out of loop
        success = True

      except Exception as e:
        retry += 1
        print("---------------------------------------------------------------------------------------")
        print(f"Error executing SQL statement for customer {customer_id}: {sql}")
        print("---------------------------------------------------------------------------------------")

        if retry > 5:
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          print(f"Error executing SQL statement for customer {customer_id} after 5 retries")
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          break # Skip this customer

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Error generating marketing insights for customer {customer_id}: {e}")
      print("---------------------------------------------------------------------------------------")

      if retry > 5:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"Error generating marketing insights for customer {customer_id} after 5 retries")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        break # Skip this customer





### Step 5: Generate customer_text_summary

#### Define Text Summary Generator Function

In [ ]:
def GenerateCustomerSummary(customer_id, customer_profile_data, data_beans_profile_data, generated_marketing_insights):
  # For each row in customer_marketing_profile, send this prompt:
  prompt = f"""
  You will be given three JSON objects with data about a customer: customer_profile_data, data_beans_profile_data, generated_marketing_insights.

  Your task is to generate a summary for this customer, highlighting the most important things a Marketing Analyst should know in order to successfully sell to the customer.

  Here is the customer's customer_profile_data:
  {customer_profile_data}

  Here is the customer's data_beans_profile_data:
  {data_beans_profile_data}

  Here is the customer's generated_marketing_insights:
  {generated_marketing_insights}

  Format your output as a text string, not JSON.

  Now think step by step and generate the summary.

  """

  response_schema = {
      "type": "object",
      "properties": {
          "output_string": {
              "type": "string"
          }
      },
      "required": "output_string"
  }


  result = GeminiLLM(prompt, response_schema=response_schema)
  return result


#### Run Text Summary Generator Function

In [ ]:
sql = """
SELECT *
  FROM `chocolate_ai.customer_marketing_profile`
 WHERE customer_profile_data        IS NOT NULL
   AND data_beans_profile_data      IS NOT NULL
   AND generated_marketing_insights IS NOT NULL
   AND customer_text_summary        IS NULL
"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_id = row.customer_id
  customer_profile_data = row.customer_profile_data
  data_beans_profile_data = row.data_beans_profile_data
  generated_marketing_insights = row.generated_marketing_insights


  retry = 0
  success = False
  while not success:
    try:
      result = GenerateCustomerSummary(customer_id, customer_profile_data, data_beans_profile_data, generated_marketing_insights)
      print(result)
      result = json.loads(result)

      # Save to database
      try:
        sql=f"""UPDATE `chocolate_ai.customer_marketing_profile`
                SET customer_text_summary = '{result["output_string"]}'
                WHERE customer_id = {customer_id};""".replace("'","\'")

        print(sql)
        RunQuery(sql)

        # Jump out of loop
        success = True

      except Exception as e:
        retry += 1
        print("---------------------------------------------------------------------------------------")
        print(f"Error executing SQL statement for customer {customer_id}: {sql}")
        print("---------------------------------------------------------------------------------------")

        if retry > 5:
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          print(f"Error executing SQL statement for customer {customer_id} after 5 retries")
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
          break # Skip this customer

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Error generating marketing insights for customer {customer_id}: {e}")
      print("---------------------------------------------------------------------------------------")

      if retry > 5:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"Error generating marketing insights for customer {customer_id} after 5 retries")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        break # Skip this customer